In [1]:
import os
import sys
sys.path.append(os.path.abspath('../../../..'))

import numpy as np
import pandas as pd

import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list
from scipy.stats import zscore
# # step one, dannce alignment


# from utlis.corr_utlis.processed_syned_load import load_filtered_data_from_h5

rec_path = '/data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_25/20241002PMCr2_17_05'

hdf5_file_path = os.path.join(rec_path, 'MIR_Aligned/aligned_predictions_with_ca_and_dF_F.h5')

# Load the DataFrame from the HDF5 file
df_merged_with_dF_F = pd.read_hdf(hdf5_file_path, key='df')

# Assuming df_merged_with_dF_F is already loaded
# Extract columns that correspond to neurons
neuron_columns = [col for col in df_merged_with_dF_F.columns if col.startswith('dF_F_roi')]
neuron_activity = df_merged_with_dF_F[neuron_columns].values  # shape: (timepoints, neurons)

# Transpose to get shape: (neurons, timepoints)
neuron_activity = neuron_activity.T

# Step to drop low-variance neurons (reduce false positives)
neuron_variances = np.var(neuron_activity, axis=1)  # Variance for each neuron
threshold = np.percentile(neuron_variances, 5)      # Keep top 95% variance
high_variance_indices = neuron_variances > threshold  # Mask for neurons to keep

# Filter neuron activity and neuron column names
neuron_activity_filtered = neuron_activity[high_variance_indices, :]
filtered_neuron_columns = [col for i, col in enumerate(neuron_columns) if high_variance_indices[i]]

# Optional: Z-score normalization along each neuron's time course
neuron_activity_normalized = zscore(neuron_activity_filtered, axis=1)
# 'correlation' is often a good metric for neural activity, but you can try 'euclidean'.
# 'average' linkage is a common choice, but feel free to experiment (e.g., 'ward', 'complete').
# Perform hierarchical clustering on the filtered and normalized data
Z = linkage(neuron_activity_normalized, method='average', metric='correlation')

df_new = df_merged_with_dF_F.copy()
df_new = df_new.reset_index()
time = df_new['timestamp_ms_mini']

head_keypoints = [1, 2, 3, 4]  # Example: EarL, EarR, Snout, SpineF
head_coords = df_merged_with_dF_F[[f"kp{idx}_{axis}" for idx in head_keypoints for axis in ['x', 'y', 'z']]].copy()


if 'com_x' in df_merged_with_dF_F.columns and 'com_y' in df_merged_with_dF_F.columns and 'com_z' in df_merged_with_dF_F.columns:
    head_coords_ego = df_merged_with_dF_F[
        [f"kp{idx}_{axis}" for idx in head_keypoints for axis in ['x', 'y', 'z']]
    ].copy()

    for axis in ['x', 'y', 'z']:
        com_col = f'com_{axis}'
        if com_col in df_merged_with_dF_F.columns:
            head_coords_ego[[f"kp{idx}_{axis}" for idx in head_keypoints]] -= df_merged_with_dF_F[com_col].values[:, None]
        else:
            print(f"Warning: {com_col} not found. Cannot adjust for egocentric {axis} coordinates.")
else:
    print("COM columns (com_x, com_y, com_z) not found. Using absolute coordinates.")




def connectivity(path: str, skeleton_name: str):
    """ 
    MIR QI TAKEN/Adapted FROM JOSH WU, FROM DAPPY/NEUROPOSELIB
    DEPRECATED

    Reads in connectivity from skeleton.py file

    Parameters
    ----------
    path : str
        Path to skeleton/connectivity Python file.
    skeleton_name : str
        Name of skeleton type to load in.

    Returns
    -------
    connectivity: Connectivity object
        Connectivity class object containing designated skeleton information
    """
    if path.endswith(".py"):
        import importlib.util

        mod_spec = importlib.util.spec_from_file_location("connectivity", path)
        con = importlib.util.module_from_spec(mod_spec)
        mod_spec.loader.exec_module(con)

        joint_names = con.JOINT_NAME_DICT[skeleton_name]  # joint names
        colors = con.COLOR_DICT[skeleton_name]  # color to be plotted for each linkage
        links = con.CONNECTIVITY_DICT[skeleton_name]  # joint linkages
        angles = con.JOINT_ANGLES_DICT[skeleton_name]  # angles to calculate

    # connectivity = Connectivity(
    #     joint_names=joint_names, colors=colors, links=links, angles=angles
    # )

    connectivity = {
            "joint_names": joint_names,
            "colors": colors,
            "links": links,
            "angles": angles,
        }

    return connectivity


def compute_mean_connection_lengths(df, connectivity_dict, offset=1):
    """
    Computes the average length of specified connections for each row in df,
    and returns the result in a dictionary format.

    Parameters
    ----------
    df : pd.DataFrame
        A DataFrame containing columns named like 'kp1_x', 'kp1_y', 'kp1_z', etc.
    connectivity_dict : dict
        A dictionary containing keypoint connectivity information, obtained from the `connectivity` function.
    offset : int, optional
        The value to add to the connectivity keypoints to match the DataFrame columns
        (since your DataFrame is 1-based), default is 1.

    Returns
    -------
    dict
        A dictionary where keys are connection labels (e.g., 'SpineF-Tail(base)'),
        and values are lists of computed distances for each row in df.
    """
    links = connectivity_dict.get("links", [])
    joint_names = connectivity_dict.get("joint_names", [])

    # Create connection labels based on joint names
    connection_labels = [
        f"{joint_names[kpA]}-{joint_names[kpB]}" for kpA, kpB in links
    ]

    # Initialize dictionary to store computed distances
    connection_distances = {label: [] for label in connection_labels}

    for _, row in df.iterrows():
        for (kpA, kpB), label in zip(links, connection_labels):
            # Convert from 0-based connectivity to 1-based column naming
            kpA_idx = kpA + offset
            kpB_idx = kpB + offset

            # Extract coordinates for the two keypoints
            xA, yA, zA = row[f'kp{kpA_idx}_x'], row[f'kp{kpA_idx}_y'], row[f'kp{kpA_idx}_z']
            xB, yB, zB = row[f'kp{kpB_idx}_x'], row[f'kp{kpB_idx}_y'], row[f'kp{kpB_idx}_z']

            # Compute Euclidean distance
            dist = np.sqrt((xB - xA) ** 2 + (yB - yA) ** 2 + (zB - zA) ** 2)
            connection_distances[label].append(dist)

    return connection_distances


def compute_joint_angles(df, connectivity_dict, offset=1):
    """
    Computes the angles formed by keypoint triplets specified in the connectivity dictionary
    for each row in the DataFrame, returning the results in a dictionary format.

    Parameters
    ----------
    df : pd.DataFrame
        A DataFrame containing columns named like 'kp1_x', 'kp1_y', 'kp1_z', etc.
    connectivity_dict : dict
        A dictionary containing keypoint connectivity information, including joint angles.
    offset : int, optional
        The value to add to the connectivity keypoints to match the DataFrame columns
        (since your DataFrame is 1-based), default is 1.

    Returns
    -------
    dict
        A dictionary where keys are angle labels (e.g., 'SpineF-SpineM-Tail(base)'),
        and values are lists of computed angles for each row in df.
    """
    angles_list = connectivity_dict.get("angles", [])
    joint_names = connectivity_dict.get("joint_names", [])

    # Create angle labels based on joint names
    angle_labels = [
        f"{joint_names[kpA]}-{joint_names[kpB]}-{joint_names[kpC]}" for kpA, kpB, kpC in angles_list
    ]

    # Initialize dictionary to store computed angles
    angle_values = {label: [] for label in angle_labels}

    for _, row in df.iterrows():
        for (kpA, kpB, kpC), label in zip(angles_list, angle_labels):
            # Convert from 0-based connectivity to 1-based column naming
            kpA_idx, kpB_idx, kpC_idx = kpA + offset, kpB + offset, kpC + offset

            # Extract coordinates for the three keypoints
            A = np.array([row[f'kp{kpA_idx}_x'], row[f'kp{kpA_idx}_y'], row[f'kp{kpA_idx}_z']])
            B = np.array([row[f'kp{kpB_idx}_x'], row[f'kp{kpB_idx}_y'], row[f'kp{kpB_idx}_z']])
            C = np.array([row[f'kp{kpC_idx}_x'], row[f'kp{kpC_idx}_y'], row[f'kp{kpC_idx}_z']])

            # Compute vectors
            v1 = A - B
            v2 = C - B

            # Normalize vectors
            v1_u = v1 / np.linalg.norm(v1)
            v2_u = v2 / np.linalg.norm(v2)

            # Compute angle using dot product and arccos
            dot_product = np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)
            angle = np.arccos(dot_product)  # Angle in radians

            # Convert to degrees for better interpretation
            angle_degrees = np.degrees(angle)
            angle_values[label].append(angle_degrees)

    return angle_values


skeleton_info = connectivity("./skeletons.py", "mouse22_notail")

connection_distances = compute_mean_connection_lengths(df_merged_with_dF_F, skeleton_info)
# Calculate the mean for each connection length
length_means = {connection: np.mean(distances) for connection, distances in connection_distances.items()}

# # Print the mean values for connection lengths
# for connection, mean_length in length_means.items():
#     print(f"Mean length for {connection}: {mean_length:.2f}")

connection_anges = compute_joint_angles(df_merged_with_dF_F, skeleton_info)

# Calculate the mean for each joint angle
angle_means = {label: np.mean(values) for label, values in connection_anges.items()}

# Print the mean values
# for label, mean_angle in angle_means.items():
#     print(f"Mean angle for {label}: {mean_angle:.2f} degrees")

Mean length for EarL-EarR: 30.87
Mean length for EarR-Snout: 31.04
Mean length for EarL-Snout: 32.45
Mean length for EarL-SpineF: 17.58
Mean length for EarR-SpineF: 17.95
Mean length for Snout-SpineF: 33.56
Mean length for SpineF-SpineM: 20.90
Mean length for SpineM-Tail(base): 33.40
Mean length for ForepawL-WristL: 4.27
Mean length for WristL-ElbowL: 14.38
Mean length for ElbowL-ShoulderL: 11.72
Mean length for ShoulderL-SpineF: 9.16
Mean length for ForepawR-WristR: 4.60
Mean length for WristR-ElbowR: 15.99
Mean length for ElbowR-ShoulderR: 11.72
Mean length for ShoulderR-SpineF: 10.69
Mean length for HindpawL-AnkleL: 12.91
Mean length for AnkleL-KneeL: 17.22
Mean length for KneeL-SpineM: 21.69
Mean length for HindpawR-AnkleR: 11.60
Mean length for AnkleR-KneeR,: 16.83
Mean length for KneeR,-SpineM: 22.56
Mean angle for Snout-EarR-SpineF: 81.99 degrees
Mean angle for Snout-EarL-SpineF: 77.64 degrees
Mean angle for EarR-SpineF-SpineM: 100.71 degrees
Mean angle for EarL-SpineF-SpineM: 1

In [7]:
# Initialize the skeleton points dictionary with SpineM as the origin
skeleton_points = {skeleton_info["joint_names"][4]: np.array([0, 0, 0])}  # 'SpineM' at origin
placed_joints = set(['SpineM'])

# Function to place keypoints dynamically
def place_keypoints(skeleton_info, length_means):
    while len(placed_joints) < len(skeleton_info["joint_names"]):
        for kpA_idx, kpB_idx in skeleton_info["links"]:
            kpA = skeleton_info["joint_names"][kpA_idx]
            kpB = skeleton_info["joint_names"][kpB_idx]

            if kpA in skeleton_points and kpB not in skeleton_points:
                length = length_means.get(f"{kpA}-{kpB}", 10)  # Default length if not found
                direction = np.random.rand(3) - 0.5  # Random direction for visualization
                direction /= np.linalg.norm(direction)  # Normalize direction
                skeleton_points[kpB] = skeleton_points[kpA] + length * direction
                placed_joints.add(kpB)

            elif kpB in skeleton_points and kpA not in skeleton_points:
                length = length_means.get(f"{kpB}-{kpA}", 10)
                direction = np.random.rand(3) - 0.5
                direction /= np.linalg.norm(direction)
                skeleton_points[kpA] = skeleton_points[kpB] + length * direction
                placed_joints.add(kpA)

place_keypoints(skeleton_info, length_means)

# Visualization
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the skeleton structure based on computed keypoints
for kpA_idx, kpB_idx in skeleton_info["links"]:
    kpA, kpB = skeleton_info["joint_names"][kpA_idx], skeleton_info["joint_names"][kpB_idx]
    if kpA in skeleton_points and kpB in skeleton_points:
        pointA, pointB = skeleton_points[kpA], skeleton_points[kpB]
        ax.plot([pointA[0], pointB[0]], [pointA[1], pointB[1]], [pointA[2], pointB[2]], 'bo-')

# Label each keypoint
for keypoint, coord in skeleton_points.items():
    ax.text(coord[0], coord[1], coord[2], keypoint)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Mouse Skeleton Visualization')
plt.show()

KeyboardInterrupt: 